<a href="https://colab.research.google.com/github/oyh1214/kpu-comp-2021/blob/master/Categorizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle # pickle을 이용하면 int 값을 그대로 저장하고 그대로 불러옴
from konlpy.tag import Okt # 형태소 분리기 임포트
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X_train, X_test, Y_train, Y_test = np.load(
    '10x10_data_max_13_size_23583_2.npy',
    allow_pickle=True)

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 49.5 MB/s 


In [8]:
pd.set_option('display.unicode.east_asian_width',True)
df = pd.read_csv('project_410x10(kor2).csv', index_col=0)
print(df.head())
print(df.info())
 
# 중복 행 확인
col_dup = df['title'].duplicated()
#print(col_dup)
 
sum_dup = df.title.duplicated().sum()
#print(sum_dup)

# title 컬럼 기준으로 중복 제거 (row 통째로 제거)
df = df.drop_duplicates(subset=['title'])
 
sum_dup = df.title.duplicated().sum()

# 인덱스 새로 고침
df.reset_index(drop=True, # False로 줄 시 기존 인덱스를 colum으로 올림
               inplace=True)

                                     title   category
0           원목 접이식 의자형 테이블   개  Furniture
1           원목 접이식 의자형 테이블   개  Furniture
2     AB스토리지 코튼 브라운 행거커버 하프  Furniture
3   AB스토리지 코튼 브라운 행거커버 베이직  Furniture
4  AB스토리지 코튼 브라운 행거커버 올인원S  Furniture
<class 'pandas.core.frame.DataFrame'>
Int64Index: 97200 entries, 0 to 97199
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     97167 non-null  object
 1   category  97200 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB
None


In [9]:
##중간 삽입 부분
df = df.dropna(axis=0)
df.info()
df.reset_index(drop=True, inplace=True)

X = df['title']
Y = df['category']
encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)
label = encoder.classes_
onehot_Y = to_categorical(labeled_Y)

okt = Okt()
for i in range(len(X)):
    X[i] = okt.morphs(X[i])

stopwords = pd.read_csv('stopwords.csv')

for i in range(len(X)):
    result = []
    for j in range(len(X[i])):
        if len(X[i][j]) > 1:
            if X[i][j] not in list(stopwords['stopword']):
                result.append(X[i][j])
    X[i] = ' '.join(result)

token = Tokenizer()
token.fit_on_texts(X)
tokened_X = token.texts_to_sequences(X)

wordsize = len(token.word_index) +1
max = 0
for i in range(len(tokened_X)):
    if max < len(tokened_X[i]):
        max = len(tokened_X[i])

X_pad = pad_sequences(tokened_X,max)
print(X_pad[:9])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82475 entries, 0 to 82475
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     82475 non-null  object
 1   category  82475 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


[[   0    0    0    0    0    0    0    0    0   23  181   80   51]
 [   0    0    0    0    0    0 3418 1131  137  142  133   16  437]
 [   0    0    0    0    0    0 3418 1131  137  142  133   16   55]
 [   0    0    0    0    0    0 3418 1131  137  142  133   16 1016]
 [   0    0    0    0    0    0    0 3418 1131  137  142   78   33]
 [   0    0    0    0    0    0 3418 1131  137  142   78   33  719]
 [   0    0    0    0    0    0    0 3418 1131  137  142 1648  205]
 [   0    0    0    0    0    0    0    0    0    0    0 8108   37]
 [   0    0    0    0    0    0    0    0    0    0 8108   37    2]]


In [10]:
model = load_model('tbt_classfication_0.9516246318817139.h5')

In [11]:
predict = model.predict(X_pad)

In [12]:
Category = ['Furniture','Life','Deco','Kitchen','Food','FasihonClothing','FasionGoods','Beauty','Jewelry']
Category_num = ['121','120','122','112','119','117','116','118','125']   
result = np.argmax(predict[0])

In [13]:
predict_category = []
for pred in predict:
    predict_category.append(Category[np.argmax(pred)])

In [14]:
df['predict'] = predict_category

In [15]:
def test_from_input_product_name():
    while (1) :
        input_product_name = input()

        okt = Okt()
        input_product_name = okt.morphs(input_product_name)

        token = Tokenizer()
        token.fit_on_texts(input_product_name)
        tokened_X1 = token.texts_to_sequences(input_product_name)

        X_pad1 = pad_sequences(tokened_X1,max)

        pred = model.predict(X_pad1)
        #print(pred)

        result = np.argmax(pred[0])
        #print(result)
        print("예상되는 상품태그는 : " , Category[result])

In [18]:
test_from_input_product_name()

닥터지 레드 블레미쉬 수딩 토너 저자극 수분 진정 토너
예상되는 상품태그는 :  Deco


KeyboardInterrupt: ignored